In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [3]:
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import os
import shutil
from zipfile import ZipFile

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [4]:
path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/"

with ZipFile(path + "train.zip") as train_zip:
    train_zip.extractall("/kaggle/working")
    
with ZipFile(path + "test.zip") as test_zip:
    test_zip.extractall("/kaggle/working")

In [5]:
train_path = "/kaggle/working/train/"
test_path = "/kaggle/working/test/"

os.mkdir(train_path + "cat")
os.mkdir(train_path + "dog")

for file in os.listdir(train_path):
    if file.startswith("cat"):
        shutil.move(train_path + file, train_path + "cat")
    elif file.startswith("dog"):
        shutil.move(train_path + file, train_path + "dog")
    else:
        print(file + "is not cat or dog.")

In [6]:
seed = 0
IMG_SHAPE = (500, 500)

In [7]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
                train_path,
                image_size=IMG_SHAPE,
                seed = seed,
                shuffle=True,
                subset="training",
                validation_split=0.1,
                batch_size=32)

Found 25000 files belonging to 2 classes.
Using 22500 files for training.


In [8]:
valid_dataset = keras.preprocessing.image_dataset_from_directory(
                train_path,
                shuffle=True,
                subset="validation",
                validation_split=0.1,
                seed=seed,
                image_size=IMG_SHAPE,
                batch_size=32)

Found 25000 files belonging to 2 classes.
Using 2500 files for validation.


In [9]:
test_dataset = keras.preprocessing.image_dataset_from_directory(test_path,
               shuffle=False,
               label_mode=None,
               image_size=IMG_SHAPE,
               batch_size=32)

Found 12500 files belonging to 1 classes.


In [10]:
augmentation = keras.Sequential([
    keras.layers.RandomBrightness(0.2),
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomContrast(0.2)
])

In [11]:
base_model = keras.applications.InceptionResNetV2(include_top=False, weights="imagenet", input_shape=(IMG_SHAPE) + (3, ))

219055592/219055592 [==============================] - 1s 0us/step


In [12]:
base_model.trainable = False
preprocess_input = keras.applications.inception_resnet_v2.preprocess_input
inputs = keras.layers.Input(shape=(IMG_SHAPE) + (3, ))

In [13]:
x = augmentation(inputs)
x = preprocess_input(inputs)

x = base_model(x, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(1)(x)

In [14]:
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 500, 500, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 500, 500, 3)       0         
 da)                                                             
                                                                 
 inception_resnet_v2 (Funct  (None, 14, 14, 1536)      54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d (  (None, 1536)              0         
 GlobalAveragePooling2D)                                     

In [15]:
init_epoch = 2

optimizer = keras.optimizers.Adam()
loss = keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])
model.fit(train_dataset, epochs=init_epoch, validation_data=valid_dataset)

Epoch 1/2
704/704 [==============================] - 747s 1s/step - loss: 0.0963 - accuracy: 0.9721 - val_loss: 0.0310 - val_accuracy: 0.9944
Epoch 2/2
704/704 [==============================] - 726s 1s/step - loss: 0.0290 - accuracy: 0.9947 - val_loss: 0.0219 - val_accuracy: 0.9948


In [16]:
preds = tf.sigmoid(model.predict(test_dataset)).numpy().flatten()
preds_clipped = np.clip(preds, 0.01, 0.99)

391/391 [==============================] - 371s 943ms/step


In [17]:
dic = dict(zip([int(i[21:-4]) for i in test_dataset.file_paths], preds_clipped))
dic = dict(sorted(dic.items()))

In [18]:
df = pd.DataFrame.from_dict(dic, orient='index', columns=['label'])
df.index.name = 'id'
df.head()

,label
id,
1,0.990000
2,0.990000
3,0.976197
4,0.990000
5,0.010000


In [19]:
df.to_csv("submissionRS.csv")

# CNN

In [20]:
model = keras.Sequential([
    augmentation,
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SHAPE) + (3,)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1)
])

In [21]:
model.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [22]:
init_epoch = 2
model.fit(train_dataset, epochs=init_epoch, validation_data=valid_dataset)

# Prediction
preds = tf.sigmoid(model.predict(test_dataset)).numpy().flatten()

Epoch 1/2
704/704 [==============================] - 197s 267ms/step - loss: 10.8620 - accuracy: 0.5021 - val_loss: 0.6934 - val_accuracy: 0.4968
Epoch 2/2
391/391 [==============================] - 33s 83ms/step


### Observations
- The model's accuracy did not improve significantly after 2 epochs, suggesting possible underfitting.
- Both training and validation accuracies hovered around 50%, indicating that the model may require more epochs, additional tuning, or changes in architecture.

### Next Steps
- Consider increasing the number of epochs for further training.
- Experiment with different architectures or hyperparameters.
- Apply additional techniques such as data augmentation or regularization to improve model performance.

In [23]:
# Clipping predictions
preds_clipped = np.clip(preds, 0.01, 0.99)


In [24]:
dic = dict(zip([int(i[21:-4]) for i in test_dataset.file_paths], preds_clipped))
dic = dict(sorted(dic.items()))

In [25]:
df = pd.DataFrame.from_dict(dic, orient='index', columns=['label'])
df.index.name = 'id'
df.head()
df.to_csv("submission_cnn.csv")

# pre-trained VGG16 model

In [26]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [27]:
base_model.trainable = False

In [28]:
model = keras.Sequential([
    augmentation,
    keras.layers.Lambda(lambda x: tf.image.resize(x, (IMG_SHAPE[0], IMG_SHAPE[1]))),
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

In [29]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [30]:
model.fit(train_dataset, epochs=5, validation_data=valid_dataset)
preds = model.predict(test_dataset)


Epoch 1/5
704/704 [==============================] - 640s 877ms/step - loss: 0.2197 - accuracy: 0.9163 - val_loss: 0.0751 - val_accuracy: 0.9700
Epoch 2/5
704/704 [==============================] - 612s 869ms/step - loss: 0.1502 - accuracy: 0.9379 - val_loss: 0.0555 - val_accuracy: 0.9760
Epoch 3/5
704/704 [==============================] - 613s 871ms/step - loss: 0.1313 - accuracy: 0.9462 - val_loss: 0.0555 - val_accuracy: 0.9784
Epoch 4/5
704/704 [==============================] - 614s 872ms/step - loss: 0.1291 - accuracy: 0.9469 - val_loss: 0.0575 - val_accuracy: 0.9776
Epoch 5/5
391/391 [==============================] - 311s 797ms/step


## **The highest validation accuracy achieved during the training of the model is 98.04% at the end of the fifth epoch.**

In [31]:
dic = dict(zip([int(i[21:-4]) for i in test_dataset.file_paths], preds.flatten()))
dic = dict(sorted(dic.items()))

In [32]:
df = pd.DataFrame.from_dict(dic, orient='index', columns=['label'])
df.index.name = 'id'
df.head()
df.to_csv("submission_vgg16.csv")